# Pretrained Convnets
Da es rechnerisch recht teuer ist, Convnets zu trainieren, lassen sich auch Teile bereits trainierter Modelle nutzen, um Probleme zu lösen. Dabei gibt es zwei Möglichkeiten:

## Feature Extraction
Hier werden die breits trainierten Schichten eines anderen Modells genutzt, um ein ähnliches Problem zu lösen. Konnte bspw. das Modell zuvor verschiedene Tiere erkennen, so wird es auch für dich nützliche Repeäsentationen enthalten, die für deine Hunde-Katzen-Klassifizierung von Bedeutung sind und die du deshalb nutzen kannst.

Auf dieser Basis werden neue dicht vernetzte Schichten gesetzt, die auf zwei Wegen trainiert werden können.
1. Du erstellst einen neuen Datensatz aus den Conv-Schichten, mit dem dann die dicht vernetzten Schichten trainiert werden.
2. Du setzt die neuen Schichten auf die Conv-Schichten und trainierst beide zusammen.

Meist werden dabei nur die ersten Schichten genutzt, da sie die grundlegenden Muster kodieren. 

In [5]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(
    weights = 'imagenet',
    include_top = False, # vernetzte Schicht auslassen
    input_shape = (150, 150, 3) # Inputdimension anpassen
)
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

## Fine-Tuning
Hier wird ein komplettes vortrainiertes Modell genommen. Die untersten Schichten mit den grundlegenden Repräsentationen bleiben eingefroren, sind also nicht von Backpropagation betroffen.
Die obersten Schichten werden dem aktuellen Problem angepasst. Sie werden weiter trainiert, sodass sich die High-Level-Repräsentationen dem neuen Problem anpassen.
Zudem müssen die unteren Schichten eingedroren werden, weil sonst die Signale der Backpropagation im neuen Probleme so große Änderungen verursachen würde, dass deren nützliche low-level Information zerstört würde.
Wie werden sie aber eingefroren?

In [7]:
# einfrieren von Schichten: wir wollen z.B. nur die Dense-Schichten trainieren:
base_2 = VGG16(
    weights = 'imagenet',
    include_top = True, # vernetzte Schicht belassen
    input_shape = (224, 224, 3) # Inputdimension anpassen
)
base_2.summary()

for layer in base_2.layers:
    if (layer.name == 'predictions') or (layer.name == 'fc2'):
        layer.trainable = True
    else:
        layer.trainable = False

553467904/553467096 [==============================] - 234s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________________